In [4]:
# import streamlit as st
# streamlit is for later use on web app, ignore now

import yfinance as yf
import numpy as np
from scipy.stats import norm
from datetime import datetime
import pandas as pd

# Workflow for retrieve and filter option chain

1. User input ticker 
- TODO: company name should also works
2. Provide valid expiration date to choose
3. Provide any specific threshold, such as Greeks, IV or Strike Price Bound


In [5]:


# 1. Function to retrieve available expiration dates for a ticker
def get_available_expirations(ticker):
    """
    Retrieve available expiration dates for a given ticker using yfinance.
    """
    stock = yf.Ticker(ticker)
    available_expirations = stock.options  # Get list of expiration dates from yfinance
    return available_expirations


# Function to retrieve stock price for a ticker
def get_stock_price(ticker):
    """
    Fetches the current stock price for a given ticker using yfinance.
    """
    ticker_data = yf.Ticker(ticker).history(period='1d')
    if ticker_data.empty:
        raise ValueError(f"No historical data found for {ticker}.")
    return ticker_data['Close'].iloc[0]

# Black-Scholes model functions to calculate Greeks for both Call and Put options
def black_scholes_greeks(S, K, T, r, sigma, option_type='call'):
    """
    Calculate the Black-Scholes Greeks (Delta, Gamma, Theta, Vega, Rho) for call or put options.
    """
    from scipy.stats import norm
    import numpy as np
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    if option_type == 'call':
        delta = norm.cdf(d1)
        rho = K * T * np.exp(-r * T) * norm.cdf(d2) / 100
    elif option_type == 'put':
        delta = norm.cdf(d1) - 1
        rho = -K * T * np.exp(-r * T) * norm.cdf(-d2) / 100

    gamma = norm.pdf(d1) / (S * sigma * np.sqrt(T))
    theta = (-S * norm.pdf(d1) * sigma / (2 * np.sqrt(T)) - r * K * np.exp(-r * T) * norm.cdf(d2 if option_type == 'call' else -d2)) / 365
    vega = S * norm.pdf(d1) * np.sqrt(T) / 100
    
    return delta, gamma, theta, vega, rho


In [6]:



# Function to retrieve S&P 500 tickers
def get_sp500_tickers():
    """
    Get a list of S&P 500 tickers from Wikipedia.
    """
    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    sp500_table = pd.read_html(url)
    sp500_df = sp500_table[0]
    return sp500_df['Symbol'].tolist()

# Function to get QQQ tickers
def get_qqq_tickers():
    """
    Get a list of tickers for QQQ.
    """
    qqq_ticker = 'QQQ'
    qqq_holdings = yf.Ticker(qqq_ticker).history(period='1d')
    if qqq_holdings.empty:
        raise ValueError(f"No data found for {qqq_ticker}.")
    return [qqq_ticker]

In [11]:
def retrieve_option_chain(tickers, expiration_dates, iv_threshold=None, theta_price_threshold=None, option_type=None, strike_price_range=None, r=0.01):
    """
    Retrieve and filter options based on Implied Volatility (IV), absolute Theta/Price, and optionally a strike price range.
    
    Parameters:
    tickers : list : List of stock tickers.
    expiration_dates : list : List of expiration dates to check.
    iv_threshold : float : Minimum acceptable IV for filtering (high IV). Default is None (no filtering).
    theta_price_threshold : float : Minimum acceptable Theta/Price for filtering (high absolute Theta/Price). Default is None (no filtering).
    option_type : str : 'call', 'put', or None (for both). Default is None (both types).
    strike_price_range : float : Percentage range for strike price filtering (e.g., 20 for 20%). Default is None (no filtering).
    r : float : Risk-free interest rate (default is 0.01).
    
    Returns:
    result_df : DataFrame : Filtered options based on the given criteria.
    """
    result_df = pd.DataFrame()

    for ticker in tickers:
        try:
            current_price = get_stock_price(ticker)

            for expiration_date in expiration_dates:
                stock = yf.Ticker(ticker)
                options = stock.option_chain(expiration_date)

                # Filter based on user choice for calls, puts, or both
                option_types = []
                if option_type == 'call':
                    option_types.append(options.calls)
                elif option_type == 'put':
                    option_types.append(options.puts)
                else:
                    option_types = [options.calls, options.puts]
                
                filtered_options = []
                for option_data in option_types:
                    for _, row in option_data.iterrows():
                        strike_price = row['strike']
                        last_price = row['lastPrice']
                        implied_volatility = row['impliedVolatility']
                        T = (datetime.strptime(expiration_date, '%Y-%m-%d') - datetime.now()).days / 365  # Time to expiration in years

                        # Calculate Greeks using Black-Scholes model
                        delta, gamma, theta, vega, rho = black_scholes_greeks(current_price, strike_price, T, r, implied_volatility, 'call' if 'call' in row['contractSymbol'].lower() else 'put')

                        # Use absolute value of Theta for comparison
                        abs_theta = abs(theta)

                        # Calculate Theta/Price using the absolute value of Theta
                        theta_price_ratio = abs_theta / last_price if last_price > 0 else None

                        # Skip IV and Theta/Price filtering if no threshold is provided
                        if (iv_threshold is not None and implied_volatility < iv_threshold) or \
                           (theta_price_threshold is not None and theta_price_ratio is not None and theta_price_ratio < theta_price_threshold):
                            continue  # Skip this option if IV or Theta/Price is below the threshold

                        # Strike price filtering: Apply range if specified by the user
                        if strike_price_range is not None:
                            lower_bound = current_price * (1 - strike_price_range / 100)
                            upper_bound = current_price * (1 + strike_price_range / 100)
                            if not (lower_bound <= strike_price <= upper_bound):
                                continue  # Skip this option if the strike price is outside the range

                        # If it passes the filters, add the option to the results
                        filtered_options.append({
                            'Ticker': ticker.upper(),
                            'Type': 'Call' if 'call' in row['contractSymbol'].lower() else 'Put',
                            'Strike': strike_price,
                            'Last Price': last_price,
                            'Implied Volatility': implied_volatility,
                            'Theta/Price': theta_price_ratio,
                            'Delta': delta,
                            'Gamma': gamma,
                            'Theta': theta,
                            'Vega': vega,
                            'Rho': rho
                        })

                # Append filtered results to result dataframe
                if len(filtered_options) > 0:
                    df = pd.DataFrame(filtered_options)
                    df['Expiration Date'] = expiration_date
                    result_df = pd.concat([result_df, df], ignore_index=True)
        
        except Exception as e:
            print(f"Error processing {ticker}: {e}")

    return result_df


In [14]:
# Example usage of the retrieve_option_chain function
tickers = ['aapl', 'tsla']
expiration_dates = ['2024-11-01', '2024-11-08', '2024-11-15', '2024-11-22', '2024-11-29', '2024-12-20']
iv_threshold = 0.5  # Only options with IV > 30%
theta_price_threshold = 0.02  # Only options with Theta/Price > 0.02 (using absolute Theta)
option_type = 'put'  # Only put options
strike_price_range = 20  # Filter strike prices within 20% of the current price

filtered_options = retrieve_option_chain(tickers, expiration_dates, iv_threshold, theta_price_threshold, option_type, strike_price_range)
display(filtered_options)


,Ticker,Type,Strike,Last Price,Implied Volatility,Theta/Price,Delta,Gamma,Theta,Vega,Rho,Expiration Date
0,TSLA,Put,180.0,0.47,0.601566,0.146706,-0.031766,0.002847,-0.068952,0.029714,-0.002612,2024-11-01
1,TSLA,Put,185.0,0.70,0.580082,0.131809,-0.047483,0.004096,-0.092266,0.041220,-0.003909,2024-11-01
2,TSLA,Put,190.0,1.04,0.563725,0.118993,-0.071541,0.005815,-0.123753,0.056868,-0.005901,2024-11-01
3,TSLA,Put,195.0,1.58,0.553960,0.104318,-0.107506,0.008016,-0.164822,0.077038,-0.008895,2024-11-01
4,TSLA,Put,200.0,2.36,0.550175,0.090443,-0.157719,0.010518,-0.213445,0.100393,-0.013106,2024-11-01
5,TSLA,Put,205.0,3.45,0.541997,0.074702,-0.218855,0.013077,-0.257722,0.122957,-0.018263,2024-11-01
6,TSLA,Put,210.0,4.94,0.538335,0.060467,-0.293405,0.015350,-0.298706,0.143352,-0.024621,2024-11-01
7,TSLA,Put,215.0,6.90,0.535405,0.047575,-0.377138,0.017034,-0.328266,0.158221,-0.031849,2024-11-01
8,TSLA,Put,220.0,9.14,0.534551,0.037575,-0.465947,0.017853,-0.343432,0.165559,-0.039646,2024-11-01
9,TSLA,Put,225.0,11.85,0.531377,0.028693,-0.555166,0.017853,-0.340010,0.164573,-0.047620,2024-11-01


# User Cases Brainstorm
1. Protect existed portfolio
2. Maximize profit
3. Volatility Trade (harder) - neural delta strategy 

For Option buyer: 
- lower IV and theta (The option loses value more slowly as time passes)

